Purpose: Takes user inputs for an event type, month, and year, calculates a date range, filters attendees based on these inputs, and then saves the filtered results to a CSV file

In [18]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk, simpledialog
from tkcalendar import Calendar
import pandas as pd
import datetime
import os

def run_gui_application():
    root = tk.Tk()
    root.withdraw()
    root.attributes('-topmost', True)
    filepath = filedialog.askopenfilename(title="Select Attendee Database", filetypes=[("CSV files", "*.csv")])
    root.attributes('-topmost', False)

    if not filepath:
        messagebox.showinfo("Operation Cancelled", "No file selected. Exiting application.")
        root.destroy()
        return

    if not filepath.endswith('.csv'):
        messagebox.showerror("Invalid File", "Selected file is not a CSV. Please select a valid CSV file.")
        root.destroy()
        return

    try:
        attendee_db = pd.read_csv(filepath)
    except Exception as e:
        messagebox.showerror("Error", f"Failed to read the attendee database: {e}")
        root.destroy()
        return

    event_type = dropdown_input_dialog(root, "Event Type", ["401k", "ESPP"], "Choose event type:")
    if event_type.lower() == 'espp':
        month_option = dropdown_input_dialog(root, "Month Option", ["April", "October"], "Choose month:")
        year_str = simpledialog.askstring("Year", "Enter the year:", parent=root)
        if not year_str or not month_option:
            root.destroy()
            return
        year = int(year_str)
        start_date, end_date = calculate_espp_date_range(month_option, year)
    elif event_type.lower() == '401k':
        start_date, end_date = date_range_dialog(root)
    else:
        messagebox.showerror("Invalid Input", "Please enter a valid event type.")
        root.destroy()
        return

    if not start_date or not end_date:
        messagebox.showinfo("Invalid Date", "Please select valid start and end dates.")
        root.destroy()
        return

    filtered_data = filter_attendees(event_type.lower(), start_date, end_date, attendee_db)
    if not filtered_data.empty:
        output_filename = os.path.join(os.getcwd(), f"{event_type}_{start_date.replace('-', '')}-{end_date.replace('-', '')}.xlsx")
        filtered_data.to_excel(output_filename, index=False, engine='openpyxl')
        messagebox.showinfo("Success", f"Data successfully saved to {output_filename}")
    else:
        messagebox.showinfo("No Data", "No matching data found for the selected criteria.")

    root.destroy()

def dropdown_input_dialog(parent, title, options, prompt):
    dialog = tk.Toplevel(parent)
    dialog.title(title)
    dialog.geometry("300x100")
    tk.Label(dialog, text=prompt).pack(pady=10)
    var = tk.StringVar(dialog)
    dropdown = ttk.Combobox(dialog, textvariable=var, values=options, state="readonly")
    dropdown.pack()
    dropdown.bind("<<ComboboxSelected>>", lambda event: dialog.destroy())
    parent.wait_window(dialog)
    return var.get()

def date_range_dialog(parent):
    dialog = tk.Toplevel(parent)
    dialog.title("Select Date Range")
    dialog.geometry("600x600")
    tk.Label(dialog, text="Select start date:").pack(pady=10)
    start_cal = Calendar(dialog, selectmode='day', date_pattern='y-mm-dd')
    start_cal.pack(pady=10)
    tk.Label(dialog, text="Select end date:").pack(pady=10)
    end_cal = Calendar(dialog, selectmode='day', date_pattern='y-mm-dd')
    end_cal.pack(pady=10)
    start_date = end_date = None
    def save_dates():
        nonlocal start_date, end_date
        start_date = start_cal.get_date()
        end_date = end_cal.get_date()
        dialog.destroy()
    ttk.Button(dialog, text="OK", command=save_dates).pack(pady=20)
    parent.wait_window(dialog)
    return start_date, end_date

def calculate_espp_date_range(month_option, year):
    if month_option.lower() == "april":
        start_date = f"{year-1}-10-16"
        end_date = f"{year}-04-15"
    elif month_option.lower() == "october":
        start_date = f"{year}-04-16"
        end_date = f"{year}-10-15"
    return start_date, end_date

def filter_attendees(event_type, start_date, end_date, attendee_db):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    attendee_db['Event Date'] = pd.to_datetime(attendee_db['Event Date'], errors='coerce')
    filtered_data = attendee_db[
        ((attendee_db['Event Type'].str.lower() == event_type) |
         (attendee_db['Event Type'].str.lower() == "401k/espp")) &
        (attendee_db['Event Date'] >= start_date) &
        (attendee_db['Event Date'] <= end_date)
    ]
    filtered_data = filtered_data.drop_duplicates(subset=['EID Extract'], keep='first')
    return filtered_data[['EID Extract', 'Event Type', 'Event Date', 'Event ID', 'Event Count', 'Status']]

if __name__ == "__main__":
    run_gui_application()
